<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/relax_amber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://raw.githubusercontent.com/sokrypton/ColabFold/main/.github/ColabFold_Marv_Logo_Small.png" height="200" align="right" style="height:240px">

# Relax your structure using OpenMM/Amber

Easy to use protein structure relaxation using [AlphaFold2](https://www.nature.com/articles/s41586-021-03819-2) and [OpenMM](https://github.com/openmm/openmm).

This notebooks is part of ColabFold. See [GitHub](https://github.com/sokrypton/ColabFold) for other notebooks.

[Mirdita M, Schütze K, Moriwaki Y, Heo L, Ovchinnikov S, Steinegger M. ColabFold: Making protein folding accessible to all.
*Nature Methods*, 2022](https://www.nature.com/articles/s41592-022-01488-1)


In [ ]:
%%time
#@title Setup AlphaFold2 and OpenMM
import sys, os
from sys import version_info
PYTHON_VERSION = f"{version_info.major}.{version_info.minor}"

if not os.path.isfile("ALPHAFOLD_READY"):
  print("installing AlphaFold...")
  os.system("pip install alphafold-colabfold")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold")
  os.system("touch ALPHAFOLD_READY")

if not os.path.isfile("CONDA_READY"):
  print("installing conda...")
  os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh")
  os.system("bash Miniforge3-Linux-x86_64.sh -bfp /usr/local")
  os.system("mamba config --set auto_update_conda false")
  os.system("touch CONDA_READY")

if not os.path.isfile("AMBER_READY"):
  print("installing amber...")
  os.system(f"mamba install -y -q -c conda-forge openmm=8.2.0 python='{PYTHON_VERSION}' pdbfixer 2>&1 1>/dev/null")
  os.system("touch AMBER_READY")

PATH = f"/usr/local/lib/python{PYTHON_VERSION}/site-packages/"
if PATH not in sys.path:
  sys.path.insert(0, PATH)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from alphafold.relax import relax
from alphafold.relax import utils
from alphafold.common import protein, residue_constants

MODRES = {'MSE':'MET','MLY':'LYS','FME':'MET','HYP':'PRO',
          'TPO':'THR','CSO':'CYS','SEP':'SER','M3L':'LYS',
          'HSK':'HIS','SAC':'SER','PCA':'GLU','DAL':'ALA',
          'CME':'CYS','CSD':'CYS','OCS':'CYS','DPR':'PRO',
          'B3K':'LYS','ALY':'LYS','YCM':'CYS','MLZ':'LYS',
          '4BF':'TYR','KCX':'LYS','B3E':'GLU','B3D':'ASP',
          'HZP':'PRO','CSX':'CYS','BAL':'ALA','HIC':'HIS',
          'DBZ':'ALA','DCY':'CYS','DVA':'VAL','NLE':'LEU',
          'SMC':'CYS','AGM':'ARG','B3A':'ALA','DAS':'ASP',
          'DLY':'LYS','DSN':'SER','DTH':'THR','GL3':'GLY',
          'HY3':'PRO','LLP':'LYS','MGN':'GLN','MHS':'HIS',
          'TRQ':'TRP','B3Y':'TYR','PHI':'PHE','PTR':'TYR',
          'TYS':'TYR','IAS':'ASP','GPL':'LYS','KYN':'TRP',
          'CSD':'CYS','SEC':'CYS'}

def pdb_to_string(pdb_file, chains=None, models=[1]):
  '''read pdb file and return as string'''

  if chains is not None:
    if "," in chains: chains = chains.split(",")
    if not isinstance(chains,list): chains = [chains]
  if models is not None:
    if not isinstance(models,list): models = [models]

  modres = {**MODRES}
  lines = []
  seen = []
  model = 1
  for line in open(pdb_file,"rb"):
    line = line.decode("utf-8","ignore").rstrip()
    if line[:5] == "MODEL":
      model = int(line[5:])
    if models is None or model in models:
      if line[:6] == "MODRES":
        k = line[12:15]
        v = line[24:27]
        if k not in modres and v in residue_constants.restype_3to1:
          modres[k] = v
      if line[:6] == "HETATM":
        k = line[17:20]
        if k in modres:
          line = "ATOM  "+line[6:17]+modres[k]+line[20:]
      if line[:4] == "ATOM":
        chain = line[21:22]
        if chains is None or chain in chains:
          atom = line[12:12+4].strip()
          resi = line[17:17+3]
          resn = line[22:22+5].strip()
          if resn[-1].isalpha(): # alternative atom
            resn = resn[:-1]
            line = line[:26]+" "+line[27:]
          key = f"{model}_{chain}_{resn}_{resi}_{atom}"
          if key not in seen: # skip alternative placements
            lines.append(line)
            seen.append(key)
      if line[:5] == "MODEL" or line[:3] == "TER" or line[:6] == "ENDMDL":
        lines.append(line)
  return "\n".join(lines)

def relax_me(pdb_in, pdb_out, max_iterations, tolerance, stiffness, use_gpu):
  pdb_str = pdb_to_string(pdb_in)
  protein_obj = protein.from_pdb_string(pdb_str)
  amber_relaxer = relax.AmberRelaxation(
    max_iterations=max_iterations,
    tolerance=tolerance,
    stiffness=stiffness,
    exclude_residues=[],
    max_outer_iterations=3,
    use_gpu=use_gpu
  )
  relaxed_pdb_lines, _, _ = amber_relaxer.process(prot=protein_obj)
  with open(pdb_out, 'w') as f:
      f.write(relaxed_pdb_lines)

In [ ]:
#@title #Relax
#@markdown - Click the little ▶ play icon to the left to get an upload prompt.
#@markdown - After relax is done, `relaxed.pdb` will automatically download.
#@markdown - If download was blocked, click the little folder 📁 icon on the left, right-click `relaxed.pdb` and select Download!
#@markdown ----
#@markdown ## Parameters
max_iterations = 2000 #@param {type:"slider", min:0, max:5000, step:250}
#@markdown - Low values might never converge and crash and high values might also run for a long time. `0` = infinity is the AF2 default, however it might run for a long time.
tolerance = 2.39 # @param {type:"number"}
#@markdown - kcal/mol, the energy tolerance of L-BFGS.
stiffness = 10.0 # @param {type:"number"}
#@markdown - kcal/mol A^2, spring constant of heavy atom restraining potential.
#@markdown - **Note:** Descriptions and default values are taken AF2.
use_gpu = False # @param {type:"boolean"}
#@markdown - set "Runtime->Change runtime type" to a GPU in Google Colab if you activate this.

from google.colab import files
pdb_dict = files.upload()
relax_me(
    pdb_in=list(pdb_dict.keys())[0],
    pdb_out="relaxed.pdb",
    max_iterations=max_iterations,
    tolerance=tolerance,
    stiffness=stiffness,
    use_gpu=use_gpu
)
files.download(f'relaxed.pdb')